In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
import pathlib
import pickle
from pathlib import Path
from skimage.io import imread
import random
import seaborn as sns
sns.set_style('white')
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.utils.plot_funcs import plot_od, plot_train_record

In [4]:
model_path = Path('/mnt/sda1/SMLAT/training_runs/model_rotated_45_venus_long_run_700_range.pkl')

In [5]:
with open(model_path, 'rb') as f:
    chromo_model = pickle.load(f)

In [6]:
chromo_model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [7]:
chromo_model.evaluation_params['ground_truth']

[[1,
  1,
  27463.76953125,
  6984.809684753418,
  58.33728313446045,
  2466.4952158927917],
 [2,
  1,
  27067.937622070312,
  7112.922248840332,
  -639.0844702720642,
  2739.3634915351868],
 [3,
  1,
  33653.31756591797,
  7093.717613220215,
  24.959683418273926,
  2591.2692546844482],
 [4,
  1,
  34095.47546386719,
  7200.852165222168,
  78.018057346344,
  1854.9737334251404],
 [5,
  1,
  36916.861877441406,
  7260.620803833008,
  9.543609619140625,
  1242.2350645065308],
 [6,
  1,
  39081.61102294922,
  7254.732856750488,
  -373.4238922595978,
  2268.0033445358276],
 [7,
  1,
  33532.005920410156,
  7480.944480895996,
  -78.50396633148193,
  1374.5031952857971],
 [8,
  1,
  35678.65234375,
  7605.734939575195,
  -202.42498517036438,
  874.411404132843],
 [9,
  1,
  46118.79333496094,
  7574.54216003418,
  108.15801620483398,
  2262.0611786842346],
 [10,
  1,
  42884.75372314453,
  7667.521438598633,
  238.85810375213623,
  784.4618260860443],
 [11,
  1,
  46287.35534667969,
  7799.6

In [8]:
plot_train_record(chromo_model)

In [9]:
iterations = [key for key, value in chromo_model.recorder['count_loss'].items()]
count_loss = [value for key, value in chromo_model.recorder['count_loss'].items()]
loc_loss = [value for key, value in chromo_model.recorder['loc_loss'].items()]
bg_loss = [value for key, value in chromo_model.recorder['bg_loss'].items()]
cross_entropy_loss = [value for key, value in chromo_model.recorder['P_locs_error'].items()]

In [10]:
fig, ax = plt.subplots(nrows=2, ncols=2)
ax[0, 0].plot(iterations, count_loss)
ax[0, 0].set_xlabel('iterations')
ax[0, 0].set_ylabel('count loss')
ax[0, 1].plot(iterations, loc_loss)
ax[0, 1].set_xlabel('iterations')
ax[0, 1].set_ylabel('localization loss')
ax[1, 0].plot(iterations, bg_loss)
ax[1, 0].set_xlabel('iterations')
ax[1, 0].set_ylabel('noise free PSF loss')
ax[1, 1].plot(iterations, cross_entropy_loss)
ax[1, 1].set_xlabel('iterations')
ax[1, 1].set_ylabel('cross entropy loss')
plt.show()

In [11]:
from cellbgnet.analyze_eval import recognition, plot_full_img_predictions, assess, assemble_full_img_predictions, limited_matching

#### For simulated data

In [12]:
eval_img = chromo_model.evaluation_params['eval_imgs']

In [13]:
plt.figure()
plt.imshow(eval_img[0])
plt.colorbar()
plt.show()

In [14]:
fov_size = [eval_img.shape[2] * 65, eval_img.shape[1] * 65]

In [15]:
preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=eval_img,
                                             batch_size=16, use_tqdm=False,
                                             nms=True, candidate_threshold=0.2,
                                             nms_threshold=0.5, 
                                             pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                             plot_num=1,
                                             win_size=128,
                                             padding=True,
                                             start_field_pos=[0, 0],
                                             padded_background=chromo_model.evaluation_params['padded_background'])

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]


In [16]:
len(preds_tmp)

20134

##### Getting values of predicted and true x, y, z, i to plot distributions

In [17]:
ground_truth = chromo_model.evaluation_params['ground_truth']

In [18]:
len(ground_truth)

21494

In [19]:
min_int = np.percentile(np.array(ground_truth)[:, -1], 25)

In [20]:
min_int

1314.3160343170166

In [21]:
min_int = 700

In [22]:
perf_dict, matches = limited_matching(ground_truth, preds_tmp, min_int=min_int, limited_x=[0, fov_size[0]],
                                     limited_y=[0, fov_size[1]], border=450, print_res=True, tolerance=250,
                                      tolerance_ax=np.inf)

FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 20117
Recall: 0.936
Precision: 0.998
Jaccard: 93.436
RMSE_lat: 35.467
RMSE_ax: 67.629
RMSE_vol: 76.365
Jaccard/RMSE: 2.634
Eff_lat: 63.931
Eff_ax: 65.554
Eff_3d: 64.743
FN: 1379.0 FP: 32.0


In [24]:
matches.shape

(20085, 12)

### Plot distributions of true vs predicted to check for any biases on the full scale

matches is a matrix where rows correspond to a single localization,
columns are as follows. There are 12 columns, I only know what the first 8 are (x and y are in large FOV nm converted)

    1. ground truth x 
    2. ground truth y
    3. ground truth z
    4. ground truth photon counts
    5. predicted x
    6. predicted y
    7. predicted z
    8. predicted photon counts

In [25]:
ground_truth[0]

[1,
 1,
 27463.76953125,
 6984.809684753418,
 58.33728313446045,
 2466.4952158927917]

In [26]:
sampled_z = [item[4] for item in ground_truth]
sampled_ph = [item[5] for item in ground_truth]

In [27]:
gt_x, pred_x = matches[:, 0], matches[:, 4]
residual_x = (gt_x - pred_x)
gt_y, pred_y = matches[:, 1], matches[:, 5]
residual_y = (gt_y - pred_y)
gt_z, pred_z = matches[:, 2], matches[:, 6]
residual_z = (gt_z - pred_z)
gt_ph, pred_ph = matches[:, 3], matches[:, 7]
residual_ph = (gt_ph, pred_ph)

In [28]:
plt.figure()
plt.scatter(pred_z, pred_ph)
plt.xlabel('Predicted z')
plt.ylabel('Predicted photon counts')
plt.show()

In [29]:
plt.figure()
plt.scatter(gt_z, gt_ph)
plt.xlabel('Ground truth z')
plt.ylabel('Ground truth photon counts')
plt.show()

In [30]:
plt.figure()
plt.scatter(gt_z, pred_z)
plt.xlabel('Ground truth z')
plt.ylabel('Prediction z')

Text(0, 0.5, 'Prediction z')

In [31]:
plt.figure()
plt.scatter(gt_x, pred_x)
plt.xlabel('Ground truth x')
plt.ylabel('Prediction x')

Text(0, 0.5, 'Prediction x')

In [32]:
plt.figure()
plt.scatter(gt_ph, pred_ph)
plt.xlabel('Ground truth photon counts')
plt.ylabel('Prediction photon counts')

Text(0, 0.5, 'Prediction photon counts')

In [34]:
plt.figure()
plt.hist(gt_z - pred_z, bins=np.arange(-100, 100, 5))
plt.show()

In [35]:
plt.figure()
plt.hist(sampled_z, bins=np.arange(-500, 500, 10))
plt.show()

#### Predicted offsets

In [36]:
offsets_x = np.array([x[11] for x in preds_tmp])
offsets_y = np.array([x[12] for x in preds_tmp])
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].hist(offsets_x, bins=100,density=True)
ax[0].axvspan(offsets_x.min(), offsets_x.max(), color='green', alpha=0.1)
ax[0].set_xlabel('offsets_x')
ax[0].set_ylabel('counts')
ax[0].set_title(f"Mean offset_x is {np.mean(offsets_x):0.3f}, min:{np.min(offsets_x):0.3f}, max:{np.max(offsets_x):0.3f}")

ax[1].hist(offsets_y, bins=100, density=True)
ax[1].axvspan(offsets_y.min(), offsets_y.max(), color='green', alpha=0.1)
ax[1].set_xlabel('offsets_y')
ax[1].set_ylabel('counts')
ax[1].set_title(f"Mean offset_y is {np.mean(offsets_y):0.3f}, min:{np.min(offsets_y):0.3f}, max:{np.max(offsets_y):0.3f}")


fig.suptitle('Offsets predicted by the network')
plt.tight_layout()
plt.show()

In [37]:
gt_z.min(), gt_z.max()

(-499.9311864376068, 499.9898672103882)

In [38]:
plt.figure()
plt.hist(gt_z, bins=np.arange(-500, 500, 20), label='ground truth', alpha=0.5)
plt.hist(pred_z, bins=np.arange(-500, 500, 20), label='predicted', alpha=0.5)

plt.legend(loc='upper right')
plt.figure('')
plt.show()

In [39]:
plt.figure()
plt.hist(pred_z, bins=np.arange(-500, 500, 20), label='predicted')
plt.legend(loc='upper right')
plt.show()

In [40]:
plt.figure()
plt.hist(gt_ph, bins=np.arange(700, 3000, 20), label='ground truth', alpha=0.4)
plt.hist(pred_ph, bins=np.arange(700, 3000, 20), label='predicted', alpha=0.4)

plt.legend(loc='upper right')
plt.show()

In [41]:
plt.figure()
plt.hist(sampled_ph, bins=np.arange(700, 3000, 10), label='sampled during training')
plt.show()

```
List of all matches localizations for further evaluation in format: 
    1. x_gt
    2. y_gt
    3. z_gt
    4. intensity_gt,
    5. x_pred,
    6. y_pred
    7. z_pred
    8. intensity_pred
    9. nms_p
    10.x_sig
    11.y_sig
    12.z_sig
```

In [42]:
sigma_x = np.array([x[7] for x in preds_tmp])
sigma_y = np.array([x[8] for x in preds_tmp])
sigma_z = np.array([x[9] for x in preds_tmp])
sigma_ph = np.array([x[10] for x in preds_tmp])

In [43]:
fig, ax = plt.subplots(nrows=2, ncols=2)
ax[0, 0].hist(sigma_x, bins=25)
ax[0, 0].axvspan(sigma_x.min(), sigma_x.max(), color='green', alpha=0.1)
ax[0, 0].set_xlabel(r'$\sigma_x$ [nm]')
ax[0, 0].set_ylabel('counts')
ax[0, 0].set_title(f"Mean sigma_x is {np.mean(sigma_x):0.3f}, min:{np.min(sigma_x):0.3f}, max:{np.max(sigma_x):0.3f}")

ax[0, 1].hist(sigma_y, bins=25)
ax[0, 1].axvspan(sigma_y.min(), sigma_y.max(), color='green', alpha=0.1)
ax[0, 1].set_xlabel(r'$\sigma_y$ [nm]')
ax[0, 1].set_ylabel('counts')
ax[0, 1].set_title(f"Mean sigma_y is {np.mean(sigma_y):0.3f}, min:{np.min(sigma_y):0.3f}, max:{np.max(sigma_y):0.3f}")


ax[1, 0].hist(sigma_z, bins=25)
ax[1, 0].axvspan(sigma_z.min(), sigma_z.max(), color='green', alpha=0.1)
ax[1, 0].set_xlabel(r'$\sigma_z$ [nm]')
ax[1, 0].set_ylabel('counts')
ax[1, 0].set_title(f"Mean sigma_z is {np.mean(sigma_z):0.3f}, min:{np.min(sigma_z):0.3f}, max:{np.max(sigma_z):0.3f}")

ax[1, 1].hist(sigma_ph, bins=25)
ax[1, 1].axvspan(sigma_ph.min(), sigma_ph.max(), color='green', alpha=0.1)
ax[1, 1].set_xlabel(r'$\sigma_{ph}$ ADU')
ax[1, 1].set_ylabel('counts')
ax[1, 1].set_title(f"Mean sigma_ph is {np.mean(sigma_ph):0.3f}, min:{np.min(sigma_ph):0.3f}, max:{np.max(sigma_ph):0.3f}")

fig.suptitle('Sigmas predicted by the network')
plt.tight_layout()
plt.show()

#### RMSE values distributions

In [44]:
residual_x = np.abs(gt_x - pred_x)
residual_y = np.abs(gt_y - pred_y)
residual_z = np.abs(gt_z - pred_z)
residual_ph = np.abs(gt_ph - pred_ph)

In [45]:
fig, ax = plt.subplots(nrows=2, ncols=2)
ax[0, 0].hist(residual_x, bins=50)
ax[0, 0].axvspan(residual_x.min(), residual_x.max(), color='green', alpha=0.1)
ax[0, 0].set_xlabel(r'$Absoulte distance x $ [nm]')
ax[0, 0].set_ylabel('counts')
ax[0, 0].set_title(f"Mean {np.mean(residual_x):0.3f}, median: {np.median(residual_x):0.3f}, min:{np.min(residual_x):0.3f}, max:{np.max(residual_x):0.3f}")

ax[0, 1].hist(residual_y, bins=50)
ax[0, 1].axvspan(residual_y.min(), residual_y.max(), color='green', alpha=0.1)
ax[0, 1].set_xlabel(r'$Absoulte distance y $ [nm]')
ax[0, 1].set_ylabel('counts')
ax[0, 1].set_title(f"Mean dist y {np.mean(residual_y):0.3f}, median: {np.median(residual_y):0.3f}, min:{np.min(residual_y):0.3f}, max:{np.max(residual_y):0.3f}")

ax[1, 0].hist(residual_z, bins=50)
ax[1, 0].axvspan(residual_z.min(), residual_z.max(), color='green', alpha=0.1)
ax[1, 0].set_xlabel(r'$Absoulte distance z $ [nm]')
ax[1, 0].set_ylabel('counts')
ax[1, 0].set_title(f"Mean dist z {np.mean(residual_z):0.3f}, median: {np.median(residual_z):0.3f}, min:{np.min(residual_z):0.3f}, max:{np.max(residual_z):0.3f}")

ax[1, 1].hist(residual_ph, bins=50)
ax[1, 1].axvspan(residual_ph.min(), residual_ph.max(), color='green', alpha=0.1)
ax[1, 1].set_xlabel(r'$Absoulte difference Photon counts $ ADU')
ax[1, 1].set_ylabel('counts')
ax[1, 1].set_title(f"Mean diff photon counts {np.mean(residual_ph):0.3f}, median: {np.median(residual_ph):0.3f}, min:{np.min(residual_ph):0.3f}, max:{np.max(residual_ph):0.3f}")

fig.suptitle('Absolute distance btn prediction and ground truth')

plt.tight_layout()
plt.show()